In [6]:
import cv2
import matplotlib.pyplot as plt

In [7]:
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

width = 368
height = 368
inWidth = width
inHeight = height

net = cv2.dnn.readNetFromTensorflow("graph_opt.pb")
thr = 0.2

In [8]:
def poseDetector(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv2.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv2.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
            cv2.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)

    t, _ = net.getPerfProfile()

    return frame

In [11]:
input = cv2.imread("image.jpg")
output = poseDetector(input)
cv2.imshow('output', output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
video_capture = cv2.VideoCapture('180419_Boxing_20_15.mp4')
video_capture.set(3, 200)
video_capture.set(4, 200)
video_capture.set(cv2.CAP_PROP_FPS, 10)

while video_capture.isOpened():
    _, frame = video_capture.read()
    output1 = poseDetector(frame)
    cv2.imshow('Video', output1)
    if cv2.waitKey(1) == 13:
        break
video_capture.release()
cv2.destroyAllWindows()

In [16]:
video_capture1 = cv2.VideoCapture(0)
video_capture1.set(3, 200)
video_capture1.set(4, 200)
video_capture1.set(cv2.CAP_PROP_FPS, 10)

while video_capture1.isOpened():
    _, frame = video_capture1.read()
    output1 = poseDetector(frame)
    cv2.imshow('Video', output1)
    if cv2.waitKey(1) == 13:
        break
video_capture1.release()
cv2.destroyAllWindows()